# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

## Лабораторная работа 10

In [1]:
import pandas as pd
import csv
import sys

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('recipes_full.csv')
dfs = np.array_split(df, 8)

In [3]:
for i, df in enumerate(dfs):
    new_df = df[['id', 'tags', 'n_steps']]
    new_df['tags'] = new_df['tags'].str.replace('[\[\]\' ]', '', regex=True)
    new_df['tags'] = new_df['tags'].str.split(',')
    new_df = new_df.explode('tags')
    
    new_df.to_csv(f'id_tag_nsteps_{i}.csv', index=False, sep=';')

C:\Users\Drama\AppData\Local\Temp\ipykernel_15376\3981177176.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].str.replace('[\[\]\' ]', '', regex=True)
C:\Users\Drama\AppData\Local\Temp\ipykernel_15376\3981177176.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].str.split(',')
C:\Users\Drama\AppData\Local\Temp\ipykernel_15376\3981177176.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [4]:
import time

In [5]:
def mean_steps(filename):
    result = {}
    data = pd.read_csv(filename, delimiter=';')
    for tag in data['tags']:
        if not tag in result:
            result[tag] = data[data['tags']==tag]['n_steps'].mean()
    return result

In [6]:
filename = input()
start = time.time()
a = mean_steps(filename)
print(f"Обработка одного файла с помощью функции mean_steps занимает: {time.time() - start}")
a
#id_tag_nsteps_0.csv

id_tag_nsteps_5.csv
Обработка одного файла с помощью функции mean_steps занимает: 57.800676345825195


{'chocolate-chip-cookies': 3.513894826849081,
 'spinach': 4.225030574806359,
 'papaya': 3.5421372719374458,
 'turkey-burgers': 3.487380330722367,
 'hanukkah': 3.8162393162393164,
 'fathers-day': 3.478689995513683,
 'halloween-cupcakes': 3.5269512721000433,
 'low-protein': 6.3330166270783845,
 'catfish': 3.5589270008795073,
 'venezuelan': 3.5108167770419425,
 'salsas': 3.5475555555555554,
 '60-minutes-or-less': 9.429733595565885,
 'time-to-make': 9.338303392558048,
 'course': 9.32728994723312,
 'main-ingredient': 9.386575061875908,
 'cuisine': 9.316262153937505,
 'preparation': 9.387009122080611,
 'occasion': 9.21197793538219,
 'north-american': 8.299048089998763,
 'condiments-etc': 4.748865759274086,
 'side-dishes': 6.784203867702874,
 'vegetables': 8.19242441272339,
 'american': 7.668154024128243,
 'southern-united-states': 5.250808015513898,
 'easy': 7.264459136618701,
 'refrigerator': 4.752103786816269,
 'heirloom-historical': 5.0291228070175436,
 'holiday-event': 8.333383869011522,

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы иметь возможность получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


In [11]:
def join_files(results):
    res_dict = {}
    all_tags = []
    all_tags_unique = set()
    for res in results:
        all_tags.extend(list(res))
    all_tags_unique = set(all_tags)
    temp = []
    for tag in all_tags_unique:
        count = 0
        summ = 0
        for res in results:
            if tag in res:
                count += 1
                summ += res[tag]
        temp.append((tag, count, summ))
    for i in temp:
        res_dict[i[0]] = i[2]/i[1]
    return res_dict

In [11]:
start = time.time()

results = []
for i in range(8):
    results.append(mean_steps(f"id_tag_nsteps_{i}.csv"))

print(f"Обработка всех 8 файлов занимает примерно: {time.time() - start}")
join_files(results)

Обработка всех 8 файлов занимает примерно: 468.5311324596405


{nan: nan,
 'number-of-servings': 7.848857463836443,
 'asparagus': 4.047274537274003,
 'nuts': 5.502536689108273,
 'appetizers': 6.230953726886838,
 'beef-kidney': 3.497236983866153,
 'spring': 4.532772970766356,
 'tomatoes': 4.8644227716622686,
 'ontario': 3.7371140319212994,
 'cantonese': 3.563640890295793,
 'mahi-mahi': 3.536824073541834,
 'egg-free': 4.786763064867167,
 'whitefish': 3.514269832464186,
 'punch': 3.595224934308819,
 'honduran': 3.514199107507498,
 'tarts': 3.757787420865954,
 'equipment': 8.573979703215958,
 'roast-beef-comfort-food': 3.4990555866508535,
 'dinner-party': 8.233700121991642,
 'beef-crock-pot': 3.495422164523571,
 'russian': 3.700495681170118,
 'cod': 3.6568376962348292,
 'easter': 4.452968464156934,
 'super-bowl': 3.581711322493426,
 'canning': 3.9457445273796767,
 'baja': 3.5375884402114126,
 'snacks-sweet': 3.496403713874639,
 'chicken-thighs-legs': 4.145724762032948,
 'ground-beef': 5.531796145111713,
 'seasonal': 6.806117244171378,
 'greens': 4.658

In [12]:
import pandas as pd

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [7]:
import multiprocessing as mp

In [19]:
%%file process_file_.py
import pandas as pd
def mean_steps(filename):
    result = {}
    data = pd.read_csv(filename, delimiter=';')
    for tag in data['tags'].dropna():
        if not tag in result:
            result[tag] = data[data['tags']==tag]['n_steps'].mean()
    return result
def process_file(filename):
    return mean_steps(filename)

Overwriting process_file_.py


In [13]:
import process_file_

In [20]:
start = time.time()
pool = mp.Pool(processes=8)
results = pool.map(process_file_.process_file, [f"id_tag_nsteps_{i}.csv" for i in range(8)])
pool.close()
pool.join()
print(f"Обработка всех 8 файлов с использованием параллельных вычислений занимает примерно: {time.time() - start}. То есть время, довольно близкое к времени, затрачиваемому на обработку одного файла")
join_files(results)

Обработка всех 8 файлов с использованием параллельных вычислений занимает примерно: 69.85281038284302. То есть время, довольно близкое к времени, затрачиваемому на обработку одного файла


{'papaya': 3.545619991525401,
 'midwestern': 4.399057620123935,
 'pork': 6.442752809849639,
 'white-rice': 3.8468491800292854,
 'pumpkin-bread': 3.5090501246924055,
 'college': 3.511593524013422,
 'stir-fry': 4.167164930758588,
 'salsas': 3.6053987652565676,
 'no-shell-fish': 3.562686342035993,
 'beginner-cook': 6.32629801613922,
 'crusts-pastry-dough-2': 3.687978716696531,
 'cocktails': 3.619653169683436,
 'freshwater-fish': 3.834537584493437,
 'whole-chicken': 4.039457838342126,
 'penne': 3.9566767075408174,
 '4-hours-or-less': 10.065279223651007,
 'beans-side-dishes': 3.515103578211491,
 'rabbit': 3.5241548351119096,
 'poultry': 7.67373887779835,
 'main-dish': 9.442348096078904,
 'sugar-cookies': 3.53784098925935,
 'infant-baby-friendly': 3.613313638633112,
 'apples': 4.856791758952958,
 'scones': 3.8039115908453156,
 'peanut-butter': 3.5844616870595587,
 'snacks-kid-friendly': 3.500914026173997,
 'roast-beef-comfort-food': 3.4990555866508535,
 'northeastern-united-states': 4.325316

5. (*) Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 

In [21]:
%%file process_file_5.py
import pandas as pd
def mean_steps(filename):
    result = {}
    data = pd.read_csv(filename, delimiter=';')
    for tag in data['tags'].dropna():
        if not tag in result:
            result[tag] = data[data['tags']==tag]['n_steps'].mean()
    return result

def process_file(queue_in, queue_out):
    while True:
        try:
            filename = queue_in.get(timeout=1)
            result = mean_steps(filename)
            queue_out.put(result)
        except queue.Empty:
            break

Overwriting process_file_5.py


In [16]:
import process_file_5

In [17]:
import queue

In [22]:
start = time.time()
queue_in = mp.Queue()
queue_out = mp.Queue()

for i in range(8):
    queue_in.put(f"id_tag_nsteps_{i}.csv")

num_processes = mp.cpu_count() // 2
processes = [mp.Process(target=process_file_5.process_file, args=(queue_in, queue_out)) for _ in range(num_processes)]

for p in processes:
    p.start()

results = []
while len(results) < 8:
    try:
        result = queue_out.get(timeout=1)
        results.append(result)
    except queue.Empty:
        pass

for p in processes:
    p.join()

final_result = join_files(results)
print(f"Обработка всех файлов с использованием очередей для подачи файлов на вход и для извлечениия результатов: {time.time() - start}")
final_result

Обработка всех файлов с использованием очередей для подачи файлов на вход и для извлечениия результатов: 122.94463920593262


{'papaya': 3.545619991525401,
 'midwestern': 4.399057620123935,
 'pork': 6.442752809849639,
 'white-rice': 3.8468491800292854,
 'college': 3.511593524013423,
 'pumpkin-bread': 3.509050124692406,
 'stir-fry': 4.167164930758588,
 'salsas': 3.6053987652565676,
 'no-shell-fish': 3.5626863420359927,
 'beginner-cook': 6.32629801613922,
 'crusts-pastry-dough-2': 3.687978716696531,
 'cocktails': 3.619653169683436,
 'freshwater-fish': 3.8345375844934373,
 'whole-chicken': 4.039457838342126,
 'penne': 3.956676707540818,
 '4-hours-or-less': 10.065279223651007,
 'beans-side-dishes': 3.515103578211492,
 'rabbit': 3.5241548351119096,
 'poultry': 7.67373887779835,
 'main-dish': 9.442348096078904,
 'sugar-cookies': 3.5378409892593496,
 'infant-baby-friendly': 3.613313638633112,
 'apples': 4.856791758952958,
 'scones': 3.8039115908453156,
 'peanut-butter': 3.5844616870595587,
 'snacks-kid-friendly': 3.500914026173997,
 'roast-beef-comfort-food': 3.4990555866508535,
 'northeastern-united-states': 4.3253